<a href="https://colab.research.google.com/github/jini1651/PyTorch/blob/master/Sequential_MNIST_J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch             
import torch.nn as nn          
import torch.nn.functional as F  
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
torch.manual_seed(777)                                      
if device =='cuda':
    torch.cuda.manual_seed_all(777) 

In [ ]:
trans = transforms.Compose([         
    transforms.ToTensor()
])
train_data = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=trans,
                          download=True)

test_data = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=trans,
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
data_loader = DataLoader(dataset = train_data, batch_size = 10, shuffle = True) 

In [ ]:
class NET(nn.Module):
  def __init__(self):
    super(NET,self).__init__()
    self.rnn = nn.RNN(input_size=28, hidden_size=10, batch_first=True, num_layers = 1)

  def forward(self,x):
    x, _status = self.rnn(x);
    return _status

In [ ]:
net = NET().to(device)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
##TRAINING##

total_batch = len(data_loader)  #총 데이터의 개수

epochs = 10   #epoch:10
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data   
        imgs = torch.squeeze(imgs)  
        
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()   #optimizer gradient 값 초기화
        out = net(imgs)         #CNN 모델을 통해 출력값 예측
        out = torch.squeeze(out)
        loss = loss_func(out, labels)     #loss 계산 (CrossEntropy)
        loss.backward()         #gradient 계산
        optimizer.step()        #학습 모델에 반영
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    
    #loss_tracker(loss_plt, torch.Tensor([avg_cost]),torch.Tensor([epoch]))  #visdom 그래프 값 추가
print('Learning Finished!')   

[Epoch:1] cost = 1.616950511932373
[Epoch:2] cost = 1.6031103134155273


KeyboardInterrupt: ignored

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data), shuffle = True) 

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = torch.squeeze(imgs)  
        
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)   #예측
        prediction = torch.squeeze(prediction)
        
        correct_prediction = torch.argmax(prediction, 1) == label     #예측값과 정답이 일치하는지 확인
        print(torch.argmax(prediction, 1))
        
        accuracy = correct_prediction.float().mean()                  #Accuracy 계산
        print('Accuracy:', accuracy.item())

tensor([6, 6, 6,  ..., 4, 3, 2], device='cuda:0')
Accuracy: 0.5514999628067017
